In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import mediapipe

In [11]:
knuckle_model = tf.keras.models.load_model('../Training_Model1_23_6_10')
df = pd.read_csv('all_hands_with_landmark.csv')

In [12]:
df.head(4)

,Unnamed: 0.1,Unnamed: 0,path,label,landmarks
0,0,0,Hand_Images/1/63.jpg,1,"[[0.7918168306350708, 0.7815876007080078], [0...."
1,1,1,Hand_Images/1/189.jpg,1,"[[0.5782292485237122, 0.8176587224006653], [0...."
2,2,2,Hand_Images/1/77.jpg,1,"[[0.8111778497695923, 0.8097538948059082], [0...."
3,3,3,Hand_Images/1/162.jpg,1,"[[0.5877158045768738, 0.821674108505249], [0.3..."


In [13]:
import ast
data = df['landmarks'].apply(ast.literal_eval)

In [14]:
label = df['label']
label.head(4)

0    1
1    1
2    1
3    1
Name: label, dtype: int64

In [15]:
new_data = []
for i in range(len(data)):
    # print(type(data[i]))
    # print(data[i])
    temp = []
    for landmark in data[i]:
        for ele in landmark:
            temp.append(ele)
    new_data.append(temp)
data = new_data

In [19]:
result = []
for landmark in data:
    norm_landmarks = landmark
    norm_landmarks = np.array(norm_landmarks)
    landmark_tensor = tf.convert_to_tensor(norm_landmarks.reshape(1, 42), dtype=tf.float32)
    landmark_tensor = tf.expand_dims(landmark_tensor, axis=0)
    # predict
    prediction = knuckle_model.predict(landmark_tensor)
    result.append(np.argmax(prediction))

1/1 [==============================] - 0s 15ms/step


2023-06-10 19:59:19.467615: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 0s 14ms/step


In [20]:
result[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [21]:
# combine result and label into df
result_df = pd.DataFrame(result, columns=['result'])
result_df['label'] = label
result_df

,result,label
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1
...,...,...
1966,8,8
1967,8,8
1968,8,8
1969,8,8


In [22]:
# calculate accuracy
result_df['result'] = result_df['result'].astype('int')
result_df['label'] = result_df['label'].astype('int')
# compare result and label in one value
result_df['compare'] = result_df['result'] == result_df['label']
# calculate accuracy
accuracy = result_df['compare'].sum() / len(result_df)
accuracy


0.8472856418061897

In [32]:
df1 = pd.read_csv('../Assets/all_hands_with_landmark.csv')
df2 = pd.read_csv('../Assets/all_hands_with_landmark2.csv')

In [33]:
# show two df type
df1.head(4)

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,path,label,landmarks
0,0,0,0,../Assets/Hand_Images/7/63.jpg,7,"[[0.656463086605072, 0.7387614250183105], [0.6..."
1,1,1,1,../Assets/Hand_Images/7/77.jpg,7,"[[0.5969318151473999, 0.7529520988464355], [0...."
2,2,2,2,../Assets/Hand_Images/7/88.jpg,7,"[[0.5936939120292664, 0.7620826959609985], [0...."
3,3,3,3,../Assets/Hand_Images/7/89.jpg,7,"[[0.5681461095809937, 0.7292035222053528], [0...."


In [34]:
df2.head(4)

,Unnamed: 0.1,Unnamed: 0,path,label,landmarks
0,0,0,Hand_Images/1/63.jpg,1,"[[0.7918168306350708, 0.7815876007080078], [0...."
1,1,1,Hand_Images/1/189.jpg,1,"[[0.5782292485237122, 0.8176587224006653], [0...."
2,2,2,Hand_Images/1/77.jpg,1,"[[0.8111778497695923, 0.8097538948059082], [0...."
3,3,3,Hand_Images/1/162.jpg,1,"[[0.5877158045768738, 0.821674108505249], [0.3..."


In [35]:
# show datatype of two df
df1.dtypes

Unnamed: 0.2     int64
Unnamed: 0.1     int64
Unnamed: 0       int64
path            object
label            int64
landmarks       object
dtype: object

In [36]:
df2.dtypes

Unnamed: 0.1     int64
Unnamed: 0       int64
path            object
label            int64
landmarks       object
dtype: object

In [37]:
import ast
lmk1 = df1['landmarks'].apply(ast.literal_eval)
lmk2 = df2['landmarks'].apply(ast.literal_eval)

In [38]:
[type(lmk1[0]), type(lmk2[0])]

[list, list]

In [39]:
# 检查 '../Assets/all_hands_with_landmark2.csv' 中 lanmark 是否都为 21 个
for i in range(len(lmk2)):
    if len(lmk2[i]) != 21:
        print(i)

In [43]:
# 检查 '../Assets/all_hands_with_landmark.csv' 中 lanmark 是否都为 21 个
for i in range(len(lmk1)):
    if len(lmk1[i]) != 21:
        print(i)
        print(lmk1[i])

1268
[]
1283
[]
1310
[]
1390
[]
1397
[]
1415
[]


In [44]:
# 删除 '../Assets/all_hands_with_landmark.csv' 中 lanmark 不为 21 个的行，在原 df 上操作
for i in range(len(lmk1)):
    if len(lmk1[i]) != 21:
        df1.drop(i, inplace=True)

KeyError: '[1268] not found in axis'

In [45]:
# 保存 df1
df1.to_csv('../Assets/all_hands_with_landmark.csv', index=False)

# 将两个 all_hands_with_landmark.csv 合并
## 同时清洗数据

In [46]:
df1 = pd.read_csv('all_hands_with_landmark.csv')
df2 = pd.read_csv('all_hands_with_landmark_2.csv')
[len(df1), len(df2)]

[1888, 2166]

In [47]:
# 合并
df = pd.concat([df1, df2], ignore_index=True)

In [54]:
# 检查是否有长度不为 21 的 landmark
import ast
lmk = df1['landmarks'].apply(ast.literal_eval)
for i in range(len(lmk)):
    if len(lmk[i]) != 21:
        print(i)

# # 保存到 ../Assets/all_hands_with_landmark.csv
# df.to_csv('../Assets/all_hands_with_landmark.csv', index=False)

633
640
658
1334
1385
1400
1427
1507
1514
1532


In [55]:
# 删除 df 中 landmark 不为 21 的行
for i in range(len(lmk)):
    if len(lmk[i]) != 21:
        df1.drop(i, inplace=True)
# 重新索引
df1.reset_index(drop=True, inplace=True)
# 再次检查
lmk = df1['landmarks'].apply(ast.literal_eval)
for i in range(len(lmk)):
    if len(lmk[i]) != 21:
        print(i)

In [56]:
# 保存到 ../Assets/all_hands_with_landmark.csv
# df.to_csv('../Assets/all_hands_with_landmark.csv', index=False)
df1.to_csv('all_hands_with_landmark.csv', index=False)

In [57]:
len(df1)

1878